In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:5 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:11 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,586 kB]
Get:13 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packages [1,220

In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").config("spark.driver.memory", "2g").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [5]:
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [6]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')


In [7]:
spark.sql("""SELECT YEAR(date) FROM home_sales""").show()


+----------+
|year(date)|
+----------+
|      2022|
|      2021|
|      2019|
|      2019|
|      2022|
|      2019|
|      2020|
|      2019|
|      2020|
|      2021|
|      2019|
|      2021|
|      2020|
|      2021|
|      2019|
|      2021|
|      2021|
|      2019|
|      2019|
|      2019|
+----------+
only showing top 20 rows



In [8]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql("""
SELECT YEAR(date), ROUND(avg(price),2) FROM home_sales WHERE bedrooms == 4 GROUP BY YEAR(date)""").show()


+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           296363.88|
|      2019|            300263.7|
|      2020|           298353.78|
|      2021|           301819.44|
+----------+--------------------+



In [9]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT YEAR(date), ROUND(avg(price),2) FROM home_sales WHERE bedrooms == 3 AND bathrooms == 3 GROUP BY YEAR(date)""").show()



+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           292725.69|
|      2019|           287287.82|
|      2020|           294204.16|
|      2021|           294211.46|
+----------+--------------------+



In [10]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql("""
SELECT YEAR(date), ROUND(avg(price),2) 
FROM home_sales 
WHERE
bedrooms == 3 
AND bathrooms == 3 
AND floors == 2
AND sqft_living >= 2000
GROUP BY YEAR(date)""").show()



+----------+--------------------+
|year(date)|round(avg(price), 2)|
+----------+--------------------+
|      2022|           290242.99|
|      2019|           289859.14|
|      2020|           292289.09|
|      2021|           296330.96|
+----------+--------------------+



In [16]:
# 6. What is the "view" rating for homes costing more than or equal to $350,000? 
# Determine the run time for this query, and round off your answer to two decimal places.

start_time = time.time()

spark.sql("""
SELECT ROUND(avg(view),2)
FROM home_sales 
WHERE
price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 0.6484005451202393 seconds ---


In [17]:
# 7. Cache the the temporary table home_sales.
spark.sql("""
CACHE TABLE cache_home_sales OPTIONS ('storageLevel' 'DISK_ONLY') SELECT * FROM home_sales;
""").show()

AnalysisException: ignored

In [18]:
# 8. Check if the table is cached.
spark.catalog.isCached('cache_home_sales')

True

In [28]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("""
SELECT ROUND(avg(view),2) 
FROM cache_home_sales 
WHERE
price >= 350000
""").show()


print("--- %s seconds ---" % (time.time() - start_time))


--- 0.0491793155670166 seconds ---


In [22]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').parquet('p_home_sales')

In [24]:
# 11. Read the formatted parquet data.
p_df=spark.read.parquet('p_home_sales')


In [25]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('p_date_built')


In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

spark.sql("""
SELECT ROUND(avg(view),2) 
FROM p_date_built
WHERE
price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+
|round(avg(view), 2)|
+-------------------+
|              32.26|
+-------------------+

--- 0.5245754718780518 seconds ---


In [32]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable('cache_home_sales')



In [33]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('cache_home_sales')


False